In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math

In [3]:
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import torchvision
import torchvision.transforms as transforms

import PIL
from PIL import Image

from torch.utils.data.dataset import Dataset
from torchvision import transforms

import pandas as pd 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

print(device)

cpu


In [5]:
data = np.loadtxt('train-1.txt', dtype=str, delimiter = '\n')
print(data.shape)

(1984694,)


8518


In [9]:
blacks = 0
whites = 0

In [10]:
for elem in data:
    if elem[0] == 'b':
        blacks += 1
    elif elem[0] == 'w':
        whites += 1

In [41]:
blacks, whites

(1019906, 921217)

In [57]:
dataBlack = np.empty(blacks, dtype = '<U718')
dataWhite = np.empty(whites, dtype = '<U718')

In [58]:
dataBlack

array(['', '', '', ..., '', '', ''], dtype='<U718')

In [59]:
curBlack = 0
curWhite = 0

In [60]:
data.dtype

dtype('<U718')

In [61]:
for i in range(data.shape[0]):
    if data[i][0] == 'b':
        dataBlack[curBlack] = data[i]
        curBlack += 1
    elif data[i][0] == 'w':
        dataWhite[curWhite] = data[i]
        curWhite += 1

In [64]:
np.savetxt('dataBlack.txt', dataBlack, delimiter='\n', fmt="%s")

In [65]:
np.savetxt('dataWhite.txt', dataWhite, delimiter='\n', fmt="%s")

In [128]:
blackTurnBoard = np.ones(shape=(15, 15), dtype=np.int8)
whiteTurnBoard = -np.ones(shape=(15, 15), dtype=np.int8)

In [167]:
def toTurn(turn):
    letter = ord(turn[0]) - ord('a')
    num = int(turn[1:]) - 1
    print(letter, num)
    return letter, num

def toBoards(gameBad, color):
    game = gameBad.split()
    res = []
    if color == "black":
        playerBoard = blackTurnBoard
    else:
        playerBoard = whiteTurnBoard
        
    curPlayer = 1
    whiteBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    blackBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    
    for i in range(1, len(game)):
        turn = game[i]
        curBoard = np.zeros(shape=(3, 15, 15))
        if curPlayer == 1:
            blackBoard[toTurn(turn)] = 1
        else:
            whiteBoard[toTurn(turn)] = -1
        
        curBoard[1,:] = blackBoard
        curBoard[2,:] = whiteBoard
        if i % 2 == 1 and color == "black":
            curBoard[0,:] = playerBoard
            res.append(curBoard)
        elif i % 2 == 0 and color == "white":
            curBoard[0,:] = playerBoard
            res.append(curBoard)

        curPlayer *= -1
        #printBoard(curBoard)
    return np.asarray(res)

In [168]:
def printBoard(board):
    for i in range(14, -1, -1):
        for j in range(15):
            if board[1, i, j] != 0:
                print(" 1",  end='')
            elif board[2, i, j] != 0:
                print("-1", end='')
            else:
                print(" 0", end='')
        print()
    print()

In [169]:
res1 = toBoards(dataBlack[0], "black") 
res2 = toBoards(dataBlack[1], "black") 
res3 = toBoards(dataBlack[2], "black") 


print(dataBlack[0])

7 8
7 9
9 10
10 10
6 11
5 10
10 9
11 8
7 11
5 11
9 8
6 10
9 9
9 7
7 10
5 12
10 8
4 12
11 7
7 8
7 9
10 10
9 9
10 8
9 8
9 7
6 9
10 9
10 7
6 10
11 9
5 9
9 10
10 11
10 12
7 11
9 12
9 11
6 11
7 12
11 8
11 7
9 6
7 5
7 14
4 8
3 7
5 7
6 6
5 8
11 6
12 5
7 6
10 6
5 6
4 6
12 8
4 10
7 8
7 9
10 8
6 8
10 9
10 7
9 8
11 8
12 9
10 11
9 10
9 9
9 7
11 9
11 10
12 11
6 9
5 10
7 6
9 6
7 7
6 6
7 5
7 4
6 5
5 4
7 10
black h8 h9 j10 k10 g11 f10 k9 l8 h11 f11 j8 g10 j9 j7 h10 f12 k8 e12 l7


In [170]:
print(res1.shape, res2.shape, res3.shape)


(10, 3, 15, 15) (20, 3, 15, 15) (14, 3, 15, 15)


#### Black

In [171]:
fileSizeBlack = 1019906 // 103
print(fileSizeBlack)
fileSizeWhite = 921217 // 83
print(fileSizeWhite)

9902
11099


In [172]:
for curFile in range(103):
    ans = np.ndarray(shape=(0, 3, 15, 15))
    for gm in range(fileSizeBlack):
        res = toBoards(dataBlack[curFile * fileSizeBlack + gm], "black")
        np.concatenate((res, ans), axis=0)
    print(ans.shape)

7 8
7 9
9 10
10 10
6 11
5 10
10 9
11 8
7 11
5 11
9 8
6 10
9 9
9 7
7 10
5 12
10 8
4 12
11 7
7 8
7 9
10 10
9 9
10 8
9 8
9 7
6 9
10 9
10 7
6 10
11 9
5 9
9 10
10 11
10 12
7 11
9 12
9 11
6 11
7 12
11 8
11 7
9 6
7 5
7 14
4 8
3 7
5 7
6 6
5 8
11 6
12 5
7 6
10 6
5 6
4 6
12 8
4 10
7 8
7 9
10 8
6 8
10 9
10 7
9 8
11 8
12 9
10 11
9 10
9 9
9 7
11 9
11 10
12 11
6 9
5 10
7 6
9 6
7 7
6 6
7 5
7 4
6 5
5 4
7 10
7 8
7 9
10 8
6 8
10 9
9 10
10 11
10 10
11 10
9 8
6 10
9 7
9 6
5 9
12 9
13 8
10 7
11 8
9 9
11 7
6 11
7 13
11 11
11 9
9 11
7 11
7 10
5 12
10 12
7 8
7 9
7 6
6 8
6 6
5 7
4 6
5 6
5 5
7 7
5 9
9 7
10 7
6 10
6 4
7 3
9 6
7 5
6 7
5 11
4 12
11 6
9 5
10 4
4 5
6 5
4 4
4 7
7 4
7 8
7 9
10 9
6 8
5 7
6 7
6 6
6 10
9 8
7 7
5 9
5 10
4 10
6 11
6 9
4 9
7 12
3 11
3 9
4 8
5 11
6 12
5 8
7 8
7 9
10 9
6 8
5 7
6 7
6 9
9 6
7 7
11 6
5 10
4 11
5 8
5 9
4 7
7 10
7 6
9 8
9 7
10 6
10 7
6 5
2 7
3 8
7 5
7 4
6 6
7 8
7 9
10 7
6 7
6 9
9 7
5 8
4 7
5 7
5 9
10 8
6 8
10 9
10 6
7 7
6 6
7 6
4 8
11 8
7 4
9 6
7 8
7 9
10 6
9 7
10 7
10 5
9 9
10 8


IndexError: index 15 is out of bounds for axis 1 with size 15

In [66]:
class CustomDatasetFromGames(Dataset):
    def __init__(self, img_paths):
        # Read data
        self.data_info = np.load(img_paths)

        # Calculate len
        self.data_len = (self.data_info).shape[0]
        
    def __getitem__(self, index):
        # Get ndarray from index
        img_as_ndarray = self.image_arr[index]
        
        # Open image
        img_as_img = Image.fromarray(img_as_ndarray)
        
        # Reshaping
        
        # Transform image to tensor
        img_as_tensor = torch.from_numpy(np.asarray(img_as_img, dtype=np.float32))
        
        # Preproc
        img_as_tensor = img_as_tensor.unsqueeze(0)

        return img_as_tensor

    def __len__(self):
        return self.data_len


In [67]:
datasetBlack = CustomDatasetFromGames('dataBlack.txt')
trainLoaderBlack = torch.utils.data.DataLoader(dataset=data, batch_size=32, shuffle=False)

OSError: Failed to interpret file 'dataBlack.txt' as a pickle